# Get At-Risk Customers Using Business Metrics

In [ ]:
import os
from datetime import datetime
from io import BytesIO, StringIO
from pathlib import Path

import boto3
import botocore.exceptions
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown

In [ ]:
PROJ_ROOT = Path.cwd().parent

In [ ]:
assert load_dotenv(dotenv_path=PROJ_ROOT.parent / '.env')

In [ ]:
import cc_churn.costs as costs
import cc_churn.visualization as vzu

## About

Get the at-risk customers and determine how many customers should be selected in order to maximize true ROI while minimizing error in predicted ROI.

## User Inputs

In [ ]:
# R2 data bucket details
bucket_name = 'cc-churn-splits'
# # name of validation data with predictions key (file) in private R2 bucket
r2_key_val_partial = 'validation_predictions__logisticregression__'
# # name of validation data with predictions key (file) in private R2 bucket
r2_key_test_partial = 'test_predictions__logisticregression__'

# columns to load
columns = [
    'clientnum',
    'card_category',
    'total_revolv_bal',
    'total_trans_amt',
    'model_name',
    'y_pred_proba',
    'y_pred',
    'best_decision_threshold',
    'is_churned',
]

# costs
# # revenue from transactions (bank earns #% of transaction volume)
interchange_rate = 0.02
# # revenue from revolving balance (~20% interest)
apr = 0.18
# # fee revenue from credit card exposure (modeled from card type)
card_fees = {"Blue": 0, "Silver": 50, "Gold": 100, "Platinum": 200}
tenure_years = 3
discount = 0.9
# # percentage of churners who can be convinced to stay (i.e. success rate
# # of saving a churning customer)
success_rate = 0.40
# # cost of intervention to get a single customer to not churn (discounts,
# # call center time, retention offers, etc.)
intervention_cost = 50
# # cost of acquiring a new customer (Customer Acquisition Cost, CAC)
replacement_cost = 200
# # maximum number of customers that can be targeted based on client's budget
num_customers_max = 200

In [ ]:
account_id = os.getenv('ACCOUNT_ID')
access_key_id = os.getenv('ACCESS_KEY_ID')
secret_access_key = os.getenv('SECRET_ACCESS_KEY')

s3_client = boto3.client(
    's3',
    endpoint_url=f'https://{account_id}.r2.cloudflarestorage.com',
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    region_name='auto'
)

# costs
multiplier = (1 - discount**tenure_years) / (1 - discount)

In [ ]:
def pandas_read_parquet_r2(bucket_name, r2_key, columns):
    """Read parquet file from private R2 bucket."""
    s3_object = s3_client.get_object(Bucket=bucket_name, Key=r2_key)
    df = pd.read_parquet(
        BytesIO(s3_object['Body'].read()),
        columns=columns,
        dtype_backend='pyarrow',
    )
    return df


def pandas_read_filtered_parquets_r2(bucket_name, key_prefix, cols_to_load):
    """Read parquet files using partial filename from private R2 bucket."""
    s3_objects = s3_client.list_objects_v2(
        Bucket=bucket_name, Prefix=key_prefix, MaxKeys=1
    )
    assert s3_objects['ResponseMetadata']['HTTPStatusCode'] == 200
    df = pd.concat(
        [
            pandas_read_parquet_r2(
                bucket_name, obj['Key'], columns=cols_to_load
            )
            for obj in s3_objects['Contents']
        ],
        ignore_index=True,
    )
    return df


def export_df_to_r2(df, bucket_name, r2_key):
    """Export DataFrame to file in private R2 bucket, if not present."""
    try:
        s3_client.head_object(Bucket=bucket_name, Key=r2_key)
        print(f"Key {r2_key} already exists in bucket {bucket_name}")
    except botocore.exceptions.ClientError as e:
        if e.response["Error"]["Code"] == "404":
            print(f"Key {r2_key} does not exist in bucket {bucket_name}")
            buffer = BytesIO()
            df.to_parquet(
                buffer,
                index=False,
                engine='pyarrow',
                compression='gzip',
            )
            response = s3_client.put_object(
                Bucket=bucket_name, Key=r2_key, Body=buffer.getvalue()
            )
            assert response['ResponseMetadata']['HTTPStatusCode'] == 200
            print(f"Exported {len(df):,} rows to key: {r2_key}")
        elif e.response["Error"]["Code"] == "403":
            print(f"Access denied to bucket {bucket_name} or key {r2_key}")
        else:
            print(f"An unexpected error occurred: {e}")

## Load Data with Predictions

In [ ]:
%%time
from io import BytesIO

import numpy as np
import pandas as pd
import sklearn.ensemble as skens
import sklearn.metrics as mtr
import sklearn.preprocessing as pp
import sklearn.utils as skut
from IPython.display import display
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# R2 data bucket details
bucket_name = "cc-churn-splits"

# columns to load
columns = [
    "clientnum",
    "card_category",
    "total_revolv_bal",
    "total_trans_amt",
    "model_name",
    "y_pred_proba",
    "y_pred",
    "best_decision_threshold",
    "is_churned",
]

ordinal_features = [
    "income_category",
    "education_level",
]
categorical_features = [
    # 'card_category',
    "marital_status",
]
numeric_features = [
    # 'customer_age',
    # # 'dependent_count',
    "months_on_book",
    "num_products",
    "months_inactive_12_mon",
    "contacts_count_12_mon",
    "total_revolv_bal",
    # 'avg_open_to_buy',
    "total_amt_chng_q4_q1",
    # 'total_trans_amt',
    "total_trans_ct",
    "total_ct_chng_q4_q1",
    # 'avg_utilization_ratio',
]
features = numeric_features + ordinal_features + categorical_features


def pandas_read_parquet_r2_v2(s3_client, bucket_name, r2_key, columns):
    """Read parquet file from private R2 bucket."""
    s3_object = s3_client.get_object(Bucket=bucket_name, Key=r2_key)
    df = pd.read_parquet(
        BytesIO(s3_object["Body"].read()),
        columns=columns,
        dtype_backend="pyarrow",
    )
    return df


df_train = pandas_read_parquet_r2_v2(
    s3_client, bucket_name, "train_data.parquet.gzip", None
)
df_val = pandas_read_parquet_r2_v2(
    s3_client, bucket_name, "validation_data.parquet.gzip", None
)
df_test = pandas_read_parquet_r2_v2(
    s3_client, bucket_name, "test_data.parquet.gzip", None
)
df_train_val = pd.concat([df_train, df_val])

X_train = df_train.drop(columns=["is_churned"])
X_val = df_val.drop(columns=["is_churned"])
X_train_val = df_train_val.drop(columns=["is_churned"])
X_test = df_test.drop(columns=["is_churned"])

y_train = df_train["is_churned"]
y_val = df_val["is_churned"]
y_train_val = df_train_val["is_churned"]
y_test = df_test["is_churned"]

df = pd.concat([df_train, df_val, df_test])
X = df.drop(columns=["is_churned"])
y = df["is_churned"]

numeric_transformer = Pipeline(steps=[("scaler", pp.MinMaxScaler())])
categorical_transformer = Pipeline(
    steps=[("ohe", pp.OneHotEncoder(handle_unknown="ignore", drop="if_binary"))]
)
ordinal_transformer = Pipeline(
    steps=[
        (
            "oe",
            pp.OrdinalEncoder(
                categories=[
                    [
                        "Unknown",
                        "Less than $40K",
                        "$40K - $60K",
                        "$60K - $80K",
                        "$80K - $120K",
                        "$120K +",
                    ],
                    [
                        "Unknown",
                        "Uneducated",
                        "High School",
                        "College",
                        "Graduate",
                        "Post-Graduate",
                        "Doctorate",
                    ],
                ],
                handle_unknown="use_encoded_value",
                dtype=np.float64,
                unknown_value=np.nan,
            ),
        ),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("ord", ordinal_transformer, ordinal_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
    n_jobs=-1,
)

# clf = LogisticRegression(
#     class_weight='balanced', random_state=42, n_jobs=-1
# )
clf = skens.HistGradientBoostingClassifier(
    # # VERSION 1
    # max_depth=3,
    # max_bins=255,
    # l2_regularization=0.25,
    # learning_rate=0.1,
    # max_iter=250,
    # class_weight='balanced',
    # random_state=42,
    # VERSION 2
    max_depth=3,
    l2_regularization=0.25,
    class_weight="balanced",
    random_state=42,
)
# clf = skens.RandomForestClassifier(
#     n_estimators=600,
#     max_depth=3,
#     class_weight='balanced',
#     random_state=42,
#     n_jobs=-1,
# )

pipe = Pipeline([("pre", preprocessor), ("clf", clf)])

best_decision_threshold = 0.5

pipe.fit(X_train[features], y_train)
y_val_pred_proba = pipe.predict_proba(X_val)[:, 1]
y_val_pred_proba = pd.Series(
    y_val_pred_proba, name="y_pred_proba", index=X_val.index
)
y_val_pred = (
    (y_val_pred_proba >= best_decision_threshold).astype(int).rename("y_pred")
)

pipe.fit(X_train_val[features], y_train_val)
y_train_val_pred_proba = pipe.predict_proba(X_train_val)[:, 1]
y_train_val_pred_proba = pd.Series(
    y_train_val_pred_proba, name="y_pred_proba", index=X_train_val.index
)
y_train_val_pred = (
    (y_train_val_pred_proba >= best_decision_threshold)
    .astype(int)
    .rename("y_pred")
)

y_test_pred_proba = pipe.predict_proba(X_test)[:, 1]
y_test_pred_proba = pd.Series(
    y_test_pred_proba, name="y_pred_proba", index=X_test.index
)
y_test_pred = (
    (y_test_pred_proba >= best_decision_threshold).astype(int).rename("y_pred")
)

y_pred_proba = pipe.predict_proba(X)[:, 1]
y_pred_proba = pd.Series(y_pred_proba, name="y_pred_proba", index=X.index)
y_pred = (y_pred_proba >= best_decision_threshold).astype(int).rename("y_pred")

precision, recall, thresholds = mtr.precision_recall_curve(
    y_train_val, y_train_val_pred
)
df_metrics_train_val = (
    pd.DataFrame.from_dict(
        {
            "recall": mtr.recall_score(y_train_val, y_train_val_pred),
            "f2_score": mtr.fbeta_score(y_train_val, y_train_val_pred, beta=2),
            "roc_auc": mtr.roc_auc_score(y_train_val, y_train_val_pred),
            "prauc": mtr.auc(recall, precision),
        },
        orient="index",
    )
    .transpose()
    .assign(split="train+val")
)

precision, recall, thresholds = mtr.precision_recall_curve(y_test, y_test_pred)
df_metrics_test = (
    pd.DataFrame.from_dict(
        {
            "recall": mtr.recall_score(y_test, y_test_pred),
            "f2_score": mtr.fbeta_score(y_test, y_test_pred, beta=2),
            "roc_auc": mtr.roc_auc_score(y_test, y_test_pred),
            "prauc": mtr.auc(recall, precision),
        },
        orient="index",
    )
    .transpose()
    .assign(split="test")
)

metrics_list = ["prauc", "f2", "recall", "rocauc"]
df_metrics = (
    pd.concat([df_metrics_train_val, df_metrics_test], ignore_index=True)
    .assign(model_name=type(clf).__name__)
    .pivot(index="model_name", columns=["split"])
    .set_axis(
        [
            "test_recall",
            "train_recall",
            "test_f2",
            "train_f2",
            "test_rocauc",
            "train_rocauc",
            "test_prauc",
            "train_prauc",
        ],
        axis=1,
    )
    .reset_index()
    .style.set_properties(
        subset=["model_name"] + [f"test_{m}" for m in metrics_list],
        **{"background-color": "yellow", "color": "black"},
    )
)
display(df_metrics)

df_val_pred = pd.concat([df_val, y_val_pred_proba, y_val_pred], axis=1).assign(
    model_name=type(clf).__name__,
    best_decision_threshold=best_decision_threshold,
)[columns]
assert df_val_pred.isna().sum().sum() == 0

df_test_pred = pd.concat(
    [df_test, y_test_pred_proba, y_test_pred], axis=1
).assign(
    model_name=type(clf).__name__,
    best_decision_threshold=best_decision_threshold,
)[
    columns
]
assert df_test_pred.isna().sum().sum() == 0

df_all_pred = pd.concat([df, y_pred_proba, y_pred], axis=1).assign(
    model_name=type(clf).__name__,
    best_decision_threshold=best_decision_threshold,
)[columns]
assert df_all_pred.isna().sum().sum() == 0

print(
    f"Size of validation data = {len(df_val_pred):,} rows X "
    f"{df_val_pred.shape[1]:,} columns"
)
print(
    f"Size of test data = {len(df_test_pred):,} rows X "
    f"{df_test_pred.shape[1]:,} columns"
)
print(
    f"Size of all data = {len(df_all_pred):,} rows X "
    f"{df_all_pred.shape[1]:,} columns"
)

Load validation data with predictions

In [ ]:
%%time
# df_val_pred = pandas_read_filtered_parquets_r2(
#     bucket_name, r2_key_val_partial, columns
# )
print(f"Got {len(df_val_pred):,} rows of validation split predictions")
with pd.option_context('display.max_columns', None):
    display(df_val_pred)

Load test data with predictions

In [ ]:
%%time
# df_test_pred = pandas_read_filtered_parquets_r2(
#     bucket_name, r2_key_test_partial, columns
# )
print(f"Got {len(df_test_pred):,} rows of test split predictions")
with pd.option_context('display.max_columns', None):
    display(df_test_pred)

Extract best decision threshold from model predictions of the validation data

In [ ]:
best_decision_threshold = df_val_pred['best_decision_threshold'].head(1).squeeze()

Extract name of best ML model from model predictions of the validation data

In [ ]:
best_model_name = df_val_pred['model_name'].head(1).squeeze()

## Model Validation

### Costs

Calculate the true savings, expected (predicted) savings and error in predicted savings (cost) using the validation data

In [ ]:
%%time
df_costs_val, _, _ = costs.get_cost(
    df_val_pred,
    best_decision_threshold,
    interchange_rate,
    apr,
    card_fees,
    multiplier,
    success_rate,
    intervention_cost,
)
with pd.option_context('display.max_columns', None):
    display(df_costs_val)

**Notes**

1. The following columns are related to the business costs and are discussed in the [project scoping document](https://github.com/edesz/credit-card-churn/blob/main/references/01_proposal.md)
   - `interchange_rev`
   - `interest_rev`
   - `fee_rev`
   - `annual_rev`
   - `clv`
   - `success_rate`
   - `expected_savings`
   - `true_savings`
   - `cum_pred_savings`
   - `cum_true_savings`
   - `n`
   - `total_intervention_cost`
   - `ROI`
   - `ROI_pred`
   - `ROI_error`
   - `ROI_percent`
   - `ROI_percent_pred`
2. The costs are only calculated for customers that are predicted to churn since these are the customers that will be targeted. For this reason, there are fewer rows in the costs `DataFrame` (`df_costs_val`) than in the `DataFrame` with the ML model predictions for the validation data (`df_val_pred`).
3. ROI is a business-focused Key Performance Indicator (KPI). It provides a broader view of the financial efficiency of an entire investment, considering both costs and gains. So, the KPI that should be reported to the client is the expected (predicted) ROI.

### Get Optimal Number of Targeted Customers (`N`)

Plot true and predicted expected savings and ROI curves to visualize the following

1. true ROI
2. predicted ROI

using the validation data

In [ ]:
%%time
vzu.plot_roi_curves(
    df_costs_val['n'],
    df_costs_val['cum_true_savings'],
    df_costs_val['cum_pred_savings'],
    df_costs_val['ROI_percent'],
    df_costs_val['ROI_percent_pred'],
    {},
    ptitle=(
        'Excluding initial Noisy Period, ROI is Maximized after Selecting Top '
        '~190 At-Risk Customers'
    ),
    legend_loc='lower right',
    xlabel=f"Number of Predicted Churners to Contact (Top-N)",
    ylabel="Expected Net Savings ($)",
    fig_size=(12, 8),
)

**Notes**

1. The model cost is the error in predicted ROI.

Below are selected components involved in the calculation of ROI to explore the source of the initial noisy period in the ROI from the above chart

In [ ]:
cols_table = [
    'n',
    'y_pred_proba',
    'total_revolv_bal',
    'interchange_rev',
    'interest_rev',
    'fee_rev',
    'expected_savings',
    'ROI_percent',
]
with pd.option_context('display.max_rows', None):
    display(
        df_costs_val[cols_table]
        .head(25)
        .style
        .set_properties(
            subset=[
                'total_revolv_bal',
                'interchange_rev',
                'interest_rev',
                'expected_savings',
                'ROI_percent',
            ],
            **{'background-color': 'yellow', 'color': 'black'}
        )
    )

**Observations**

1. High fluctuations are caused by customers with a high `total_revolv_bal`. These customers generate higher `interest_rev` for the client than other customers. The `interest_rev` term dominates the calculation of ROI, which is not the case for other customers. As a result, ROI fluctuates strongly when these customers are included. Selecting as many **high `total_revolv_bal`** customers as possible captures steep increases in ROI.
2. `fee_rev` is fixed based on the `card_category` column. So, the interest revenue can dominate the overall ROI if the customer has a high revolving balance.

**Observations from the chart**

1. There are customers with a high `total_revolv_bal` in the top 75 customers. This results in the maximum possible ROI. Between ~75 and ~210, the influence of these customers is minimal so strong fluctuations are not observed. Between ~210 and ~245, the high `total_revolv_bal` customers appear again. After selecting the top ~245 customers, ROI shows a weak downward trend.
2. Depending on the client's available budget, the second increase resulting in a ROI peak at ~245 customers should be selected. There are two possible scenarios
   - if the budget allows for targeting at most the top 100 customers then the optimal number of customers is ~35
   - if there is room in the budget to target all possible at-risk cutomers then the optimal number of customers is ~245

   Here, we will assume the latter is true. So, the optimal number customers to be targeted is ~245.

Find the optimal number of customers to target in order to minimize error in predicted ROI and maximize true ROI, using the costs on the validation data

In [ ]:
%%time
df_costs_optimal = (
    df_costs_val
    .query(
        "(total_intervention_cost > 0) & "
        # capture second peak in ROI
        "(n >= 200) & (n < 250)"
    )
    .sort_values(
        by=['ROI', 'ROI_error', 'n'], ascending=[False, True, True],
        ignore_index=True,
    )
    .head(1)
)
optimal_N_roi = df_costs_optimal['n'].squeeze()
cols_costs = [
    'n',
    'cum_true_savings',
    'cum_pred_savings',
    'ROI_error',
    'ROI_percent',
    'ROI_percent_pred',
]
(
    df_costs_optimal[cols_costs]
    .style
    .set_properties(
        subset=['ROI_error', 'ROI_percent_pred'],
        **{'background-color': 'yellow', 'color': 'black'}
    )
)

In [ ]:
roi_error_optimal_val = df_costs_optimal['ROI_error'].squeeze()
predicted_roi_optimal_val = df_costs_optimal['ROI_percent_pred'].squeeze()
Markdown(
    "**Observations**\n"
    "1. In order to maximize true ROI and minimize the error in predicted "
    f"ROI, the optimal number of customers to target is {optimal_N_roi}. "
    "This is consistent with observations from the chart above.\n"
    f"2. If the top {optimal_N_roi} customers from the test data are "
    "targeted, then the\n"
    "   - error in the predicted ROI is approximately "
    f"{roi_error_optimal_val:.1f}%\n"
    f"   - predicted ROI is approximately {predicted_roi_optimal_val:.1f}%"
)

## Model Evaluation

### Class Imbalance

Get the true and predicted class imbalance for the test data

In [ ]:
%%time
df_true_pred_class_imbalance = (
    (
        df_test_pred['y_pred']
        .value_counts(normalize=True)
        .rename('predicted')
        .to_frame()
    )
    .merge(
        (
            df_test_pred['is_churned']
            .value_counts(normalize=True)
            .rename('true')
            .to_frame()
        ),
        left_index=True,
        right_index=True,
    )
)
df_true_pred_class_imbalance.index = df_true_pred_class_imbalance.index.map(
    {0: 'No Churn', 1: 'Churn'}
)
churn_true = df_true_pred_class_imbalance.loc['Churn']['true']
churn_pred = df_true_pred_class_imbalance.loc['Churn']['predicted']
df_true_pred_class_imbalance

In [ ]:
Markdown(
    "**Observations**\n"
    f"1. The class imbalance in the test split is approximately the same as that "
    f"in the training split, which was seen in the EDA notebook. "
    f"~{100*churn_true:.2f}% of customers showed churn in the test data.\n"
    f"2. Due to the inaccuracy of the model, ~{churn_pred:.2f}% instead of "
    f"~{churn_pred:.2f}% of customers are predicted to churn."
)

The class imbalance in the validation and test data is shown below

In [ ]:
%%time
df_val_test_true_class_imbalance = (
    df_val_pred['is_churned']
    .value_counts(normalize=True)
    .rename('validation')
    .to_frame()
    .merge(
        df_test_pred['is_churned'].value_counts(normalize=True).rename('test').to_frame(),
        left_index=True,
        right_index=True,
        how='left',
    )
)
df_val_test_true_class_imbalance.index = df_val_test_true_class_imbalance.index.map(
    {0: 'No Churn', 1: 'Churn'}
)
df_val_test_true_class_imbalance

**Notes**

1. It is reassuring that the class imbalances are nearly equivalent between the validation and test splits. Patterns seen in during model validation should not be impacted by changes of the class imbalance during model evaluation.

Show the class imbalance and distribution of prediction probabilities for the test data

In [ ]:
%%time
vzu.plot_class_imbalance_proba_distribution(
    df_clasS_imbalance=df_true_pred_class_imbalance.rename(columns=str.title),
    df_probabilities=(df_test_pred['y_pred_proba']*100),
    ptitle1='~5% Higher Churn Predicted in Test Split',
    title1_xloc=-0.3,
    ptitle2=(
        'Predicted Probabilities show Right Skew with Weak Peak Above ~90%'
    ),
    vline_label=f'Optimized Churn Cutoff ({best_decision_threshold*100:.0f}%)',
    decision_threshold=best_decision_threshold,
    subfigure_width_ratios=[1.15, 3],
    fig_size=(12, 4)
)

**Observations**

1. As expected from the predicted class imbalance, the distribution of predicted probabilities is right-skewed and a small fraction of customers have a predicted probability above 50% (the tuned classification decision threshold).

### Costs

Calculate the true savings, expected (predicted) savings and error in predicted savings (cost) using the test data

In [ ]:
%%time
df_costs_test, _, _ = costs.get_cost(
    df_test_pred,
    best_decision_threshold,
    interchange_rate,
    apr,
    card_fees,
    multiplier,
    success_rate,
    intervention_cost,
)
with pd.option_context('display.max_columns', None):
    display(df_costs_test)

**Notes**

1. The column names are identical to those in the costs calculated using the validation data (`df_costs_val`).

### Impact of Churn

The total impact of churn is estimated below for test data split

In [ ]:
%%time
df_churn_impact_test = (
    df_costs_test
    .groupby('is_churned')
    .agg({'clv': ['sum', 'count']})
    .set_axis(['clv_total', 'num_customers'], axis=1)
    .reset_index()
    .assign(
        clv_fraction=lambda df: (
            (df['clv_total']/df['clv_total'].sum()).mul(100)
        ),
        clv_per_customer=lambda df: df['clv_total']/df['num_customers'],
    )
    .query("is_churned == True")
    .reset_index(drop=True)
    .assign(
        impact_per_customer=lambda df: df['clv_per_customer']+replacement_cost,
        impact_total=lambda df: (
            df['clv_total']+df['num_customers'].mul(replacement_cost)
        ),
    )
)
impact = df_churn_impact_test['impact_total'].squeeze()
impact_per_cust = df_churn_impact_test['impact_per_customer'].squeeze()
frac_clv_lost = df_churn_impact_test['clv_fraction'].squeeze()
clv_per_customer_lost = df_churn_impact_test['clv_per_customer'].squeeze()
df_churn_impact_test

**Notes**

1. As per the assumption made in the scoping noteboo, the CAC is assumed to be $200 CAD.

In [ ]:
Markdown(
    "**Observations**\n"
    f"1. Approximately {frac_clv_lost:.1f}% of customer lifetime value is "
    "lost due to the 16% over the past 12 months in the test data split. The "
    f"impact to the client is a loss of approximately "
    f"{clv_per_customer_lost:.1f} dollars of customer lifetime value (per "
    "customer). Accounting for the cost to CAC, the total loss is "
    f"{impact_per_cust:,.1f} CAD per customer, or {impact:,.1f} CAD overall, "
    "due to churn."
)

### Get Optimal Number of Targeted Customers (`N`)

Plot true and predicted expected savings and ROI curves to visualize the following

1. true ROI
2. predicted ROI
3. optimal number of customers based on error in predicted ROI

using the test data, and compare to the optimal number of customers based on error in predicted ROI using the test data

In [ ]:
%%time
vzu.plot_roi_curves(
    df_costs_test['n'],
    df_costs_test['cum_true_savings'],
    df_costs_test['cum_pred_savings'],
    df_costs_test['ROI_percent'],
    df_costs_test['ROI_percent_pred'],
    {
        f'Optimal N (validation) = {optimal_N_roi}': {
            'x': optimal_N_roi, 'colour': 'black',
        },
        f'Optimal N (test) = {255}': {
            'x': 255, 'colour': 'red',
        },
    },
    ptitle=(
        f'Top {255:,} Customers Should be Contacted To Maximize '
        'Predicted ROI and Minimize Error in ROI'
    ),
    legend_loc='lower right',
    xlabel=f"Number of Predicted Churners to Contact (Top-N)",
    ylabel="Expected Net Savings ($)",
    fig_size=(12, 8),
)

**Observations**

1. This chart shows that 238 is not the optimal number of customers in order to maximize true ROI (green curve). The optimal ROI should be 254, which captures the second peak before the true and predicted ROI diverge.

Find the optimal number of customers to target in order to maximize true ROI, using the costs from the test data

In [ ]:
%%time
df_costs_optimal_test = (
    df_costs_test
    .query(
        "(total_intervention_cost > 0) & "
        # capture second peak in ROI
        "(n >= 200) & (n < 300)"
    )
    .sort_values(
        by=['ROI', 'ROI_error', 'n'], ascending=[False, True, True],
        ignore_index=True,
    )
    .head(1)
)
cols_costs = [
    'n',
    'cum_true_savings',
    'cum_pred_savings',
    'clv',
    'ROI_error',
    'ROI_percent',
    'ROI_percent_pred',
]
optimal_N_roi_test = df_costs_optimal_test['n'].squeeze()
(
    df_costs_optimal_test[cols_costs].assign(split='test')
    .style
    .set_properties(
        subset=['ROI_percent_pred'],
        **{'background-color': 'yellow', 'color': 'black'}
    )
)

In [ ]:
roi_error_optimal = df_costs_optimal_test['ROI_error'].squeeze()
true_roi_optimal = df_costs_optimal_test['ROI_percent'].squeeze()
predicted_roi_optimal = df_costs_optimal_test['ROI_percent_pred'].squeeze()
Markdown(
    "**Observations**\n"
    "1. In order to maximize true ROI and minimize the error in true "
    f"ROI, the optimal number of customers to target is {optimal_N_roi_test}.\n"
    f"2. If the top {optimal_N_roi_test} customers from the test data are "
    "targeted, then the\n"
    f"   - error in the predicted ROI is approximately {roi_error_optimal:.1f}%\n"
    f"   - true ROI is approximately {true_roi_optimal:.1f}%\n"
    f"   - predicted ROI is approximately {predicted_roi_optimal:.1f}%"
)

### Get Loss in Predicted ROI Using Optimized Number of Targeted Customers (`N`)

We must follow the recommendations from model validation that the top 238 predicted churners should be contacted. If we do this, then we can calculate the percent change in the KPI (predicted ROI) compared to this optimal value. This percent change is the predicted loss in ROI that will be incurred by the client by following the recommendations from the validation data

In [ ]:
%%time
df_costs_test_best = (
    df_costs_test[['n', 'y_pred_proba', 'cum_pred_savings', 'ROI_percent_pred']]
    .query(f"(n == {optimal_N_roi}) | (n == {optimal_N_roi_test})")
    .sort_values(by=['n'], ascending=True, ignore_index=True)
    .assign(
        pct_pred_ROI_lost=lambda df: (
            100*df['ROI_percent_pred'].pct_change()
        ),
        source_for_n=lambda df: pd.Series(
            ['validation data', 'test data'], index=df.index
        )
    )
)
error_pred_roi = (
    df_costs_test_best
    .query("source_for_n == 'test data'")
    ['pct_pred_ROI_lost']
    .squeeze()
)
display(
    df_costs_test_best
    .style
    .set_properties(
        subset=['cum_pred_savings', 'ROI_percent_pred', 'pct_pred_ROI_lost'],
        **{'background-color': 'yellow', 'color': 'black'}
    )
)

**Notes**

1. Calculations are performed using customers in the unseen (test) data split.

In [ ]:
Markdown(
    "**Observations**\n"
    "1. If we apply the recommendations from model validation and contact "
    f"(target) the top {optimal_N_roi} customers in unseen data (test split), "
    "then the client is incorrectly reported a loss of approximately "
    f"{error_pred_roi:.2f}% of the maximum possible predicted ROI."
)

Append column to costs indicating if targeting customer maximizes ROI

In [ ]:
df_costs_test = (
    df_costs_test
    .assign(maximizes_roi=lambda df: df['n'] <= optimal_N_roi)
)
(
    df_costs_test
    [
        [
            'clientnum',
            'n',
            'y_pred_proba',
            'y_pred',
            'clv',
            'ROI_percent_pred',
            'maximizes_roi',
        ]
    ]
)

### At-Risk Customers

In order to identify at-risk customers from the `y_pred_proba` (predicted probability) column, we must pick an optimal decision threshold based on the business goal (catching true churners). For the current business use-case, we need to prioritize recall. The optimal decision threshold was determined during ML development.

This decision threshold was optimized to maximize F2 Score, since it prioritizes recall over precision, which is in line with the business goal. The tuned threshold is stored in the `best_decision_threshold` column of `df_val_pred` and `df_test_pred`. The `y_pred` column was created by comparing `y_pred_proba` to the best decision threshold. With this in mind, the `y_pred` column already indicates if a customer is at-risk (1) or not (0).

So, the `y_pred` column will now be renamed to `is_at_risk`

In [ ]:
%%time
df_costs_test = df_costs_test.rename(columns={'y_pred': 'is_at_risk'})
with pd.option_context('display.max_columns', None):
    display(df_costs_test)

## Export Project Deliverables to Private R2 Bucket

Get the current timestamp in the format `YYmmdd_HHMMSS`

In [ ]:
curr_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

### Unseen Data (Test Split) Customers with Indicator of At-Risk and Maximizing ROI

Combine costs (predicted churners) with predicted non-churners

In [ ]:
%%time
df_test_pred_with_costs = (
    pd.concat(
        [
            df_costs_test.assign(y_pred=1),
            df_test_pred.query("y_pred != 1"),
        ],
        ignore_index=True
    )
    .fillna(
        {
            'interchange_rev': np.nan,
            'interest_rev': np.nan,
            'fee_rev': np.nan,
            'annual_rev': np.nan,
            'clv': np.nan,
            'success_rate': np.nan,
            'expected_savings': np.nan,
            'true_savings': np.nan,
            'cum_pred_savings': np.nan,
            'cum_true_savings': np.nan,
            'n': np.nan,
            'random_savings': np.nan,
            'total_intervention_cost': np.nan,
            'ROI': np.nan,
            'ROI_pred': np.nan,
            'ROI_error': np.nan,
            'ROI_percent': np.nan,
            'ROI_percent_pred': np.nan,
            'maximizes_roi': np.nan,
            'is_at_risk': 0,
        }
    )
    .convert_dtypes(dtype_backend='pyarrow')
)
with pd.option_context('display.max_columns', None):
    display(df_test_pred_with_costs)

Next, export to a file in the R2 bucket with the following file name format `test_predictions_with_business_metrics__logisticregression__<current-timestamp-YYmmdd_HHMMSS>.parquet.gzip`

In [ ]:
# %%time
# export_df_to_r2(
#     df_test_pred_with_costs,
#     bucket_name,
#     (
#         f"test_predictions_with_business_metrics__{best_model_name.lower()}__"
#         f"{curr_timestamp}.parquet.gzip"
#     ),
# )